In [2]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to your dataset
train_data_dir = 'train'
test_data_dir = 'test'

# Parameters
batch_size = 100
image_size = (128, 128)
epochs = 50
num_classes = 10  # Change this according to the number of classes in your dataset

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

# Build CNN Model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

# Save the model
model.save('fruit_classifier_model.h5')


Found 2301 images belonging to 10 classes.
Found 1025 images belonging to 10 classes.
Epoch 1/50
24/24 [==============================] - 11s 431ms/step - loss: 2.1077 - accuracy: 0.1943 - val_loss: 1.8420 - val_accuracy: 0.3063
Epoch 2/50
24/24 [==============================] - 10s 423ms/step - loss: 1.6723 - accuracy: 0.3811 - val_loss: 1.8190 - val_accuracy: 0.3424
Epoch 3/50
24/24 [==============================] - 10s 429ms/step - loss: 1.4592 - accuracy: 0.4494 - val_loss: 2.1692 - val_accuracy: 0.3454
Epoch 4/50
24/24 [==============================] - 10s 429ms/step - loss: 1.5423 - accuracy: 0.4129 - val_loss: 1.6471 - val_accuracy: 0.4068
Epoch 5/50
24/24 [==============================] - 11s 466ms/step - loss: 1.3552 - accuracy: 0.4933 - val_loss: 1.8647 - val_accuracy: 0.3493
Epoch 6/50
24/24 [==============================] - 11s 432ms/step - loss: 1.3297 - accuracy: 0.5263 - val_loss: 1.7709 - val_accuracy: 0.3990
Epoch 7/50
24/24 [==============================] - 11s 

/Users/hwaqar/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler

# Set the path to your dataset
train_data_dir = 'train'
test_data_dir = 'test'

# Parameters
batch_size = 100
image_size = (128, 128)
epochs = 50
num_classes = 10

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Build Improved CNN Model
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Learning Rate Scheduler
def lr_scheduler(epoch, lr):
    if epoch % 10 == 0 and epoch > 0:
        return lr * 0.9  # Adjust the multiplier as needed
    return lr

# Use callbacks for early stopping, model checkpoint, and learning rate scheduler
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True),
    LearningRateScheduler(lr_scheduler)
]

# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
    callbacks=callbacks
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

# Save the model
model.save('improved_fruit_classifier_model.h5')


Found 2301 images belonging to 10 classes.
Found 1025 images belonging to 10 classes.
Epoch 1/50
24/24 [==============================] - 36s 1s/step - loss: 12.5890 - accuracy: 0.2829 - val_loss: 10.0539 - val_accuracy: 0.1620 - lr: 0.0010
Epoch 2/50
24/24 [==============================] - 36s 2s/step - loss: 4.2138 - accuracy: 0.3055 - val_loss: 9.4549 - val_accuracy: 0.1424 - lr: 0.0010
Epoch 3/50
24/24 [==============================] - 37s 2s/step - loss: 2.2894 - accuracy: 0.3146 - val_loss: 9.9161 - val_accuracy: 0.1727 - lr: 0.0010
Epoch 4/50
24/24 [==============================] - 35s 1s/step - loss: 1.9632 - accuracy: 0.3568 - val_loss: 10.3140 - val_accuracy: 0.1678 - lr: 0.0010
Epoch 5/50
24/24 [==============================] - 36s 1s/step - loss: 1.7739 - accuracy: 0.4176 - val_loss: 6.1165 - val_accuracy: 0.1298 - lr: 0.0010
Epoch 6/50
24/24 [==============================] - 35s 1s/step - loss: 1.6151 - accuracy: 0.4459 - val_loss: 7.4988 - val_accuracy: 0.1229 - lr: 